In [2]:
import xgboost as xgb
import pandas as pd
from pandas import *
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.display import display, HTML
# %pylab
%matplotlib inline

In [6]:
# read data
def read(path,filename,names):
    data = pd.read_table(path+filename, names=names)
    return data
def dictFactory(data,keyName,valueName):
    dic = {}
    for i in range(0,data[keyName].size):
        dic[data[keyName][i]]=data[valueName][i]
    return dic


In [26]:
# read cluster_map from file
def readCluster():  
    clusterName = ['district_hash','district_id']
    clusterPath = './season_1/training_data/cluster_map/'
    clusterFilename = 'cluster_map'
    cluster = read(clusterPath,clusterFilename,clusterName)
    return cluster

# read orders
def readOrders(ordersFilename):
    ordersPath = './season_1/training_data/order_data/'
#     ordersFilename = 'order_data_2016-01-08'
    names = ['order_id', 'driver_id','passenger_id','start_district_hash','dest_district_hash','Price','Time']
    orders = read(ordersPath,ordersFilename,names)
    return orders

def readWeather():
    weatherPath = './season_1/training_data/weather_data/'
    weatherFilename = 'weather_data_2016-01-08'
    names = ['Time','Weather','temperature','PM2.5']
    weather = read(weatherPath,weatherFilename,names)
    return weather

In [28]:
# load data 
# orders = readOrders()
# cluster_map = readCluster()
# weather = readWeather()

In [29]:
# data must be orders
def replaceDistHash(orders):
    start_id = []
    dest_id = []
    rows = orders.shape[0]
    dic = dictFactory(cluster_map,'district_hash','district_id')
    for i in range(0,rows):
        start_id.append(dic[orders.start_district_hash[i]])
        dest_id.append(dic.get(orders.dest_district_hash[i]))
    orders['start_id'] = Series(start_id)
    orders['dest_id'] = Series(dest_id)
    orders.drop(['start_district_hash','dest_district_hash'],inplace=True,axis=1)
    return orders


In [30]:
# orders = replaceDistHash(orders)
# orders.head()

In [40]:
class Orders():
    def __init__(self,ordersFilename):
        self.orders = readOrders(ordersFilename)
        self.success = 0
        self.rows = self.orders.shape[0]
    def replaceDistHash(self):
        self.orders = replaceDistHash(self.orders)
    
    def count():
        pass
    
    def addTimeId(self):
        stamp = self.orders['Time']
        _datetime = pd.DatetimeIndex(stamp)
        _tmp = _datetime.hour * 60 + _datetime.minute
        self.orders['TimeId'] = _tmp/10
        self.orders.drop('Time',inplace=True,axis = 1)
    def addWeekday():
        
        pass
    def deleteHash(self,names):
        self.orders.drop(names,inplace=True,axis=1)
        
    def getLabel(self):
        label = {}
        driver_id = self.orders['driver_id']
        for i in range(0,self.rows):
            if driver_id.iloc[i]==driver_id.iloc[i]:
                label[i] = 1
            else:
                label[i] = 0
        self.label = DataFrame(label,index=['label']).transpose()

In [36]:
# extract feature
def prepareData(filename):
    orders = Orders(filename)
    orders.addTimeId()
    orders.replaceDistHash()
    orders.getLabel()
    orders.deleteHash(['order_id','passenger_id','driver_id'])
    return orders
# load data to xgb
def loadData(filename):
    orders = prepareData(filename)
    ordersArray = orders.orders.as_matrix()
    labelArray = orders.label.as_matrix()
    dtrain = xgb.DMatrix(ordersArray,label =labelArray)
    return dtrain


In [38]:
# load data to xgboost 

dtrain = loadData('order_data_2016-01-15')
dtest = loadData('order_data_2016-01-08')

In [47]:
param = {'max_depth':4, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)

preds = bst.predict(dtest)

In [48]:
# get error
labels = dtest.get_label()
print ('error=%f' % ( sum(1 for i in range(len(preds)) if int(preds[i]>0.5)!=labels[i]) /float(len(preds))))

error=0.142761
